Pre-process SVI Data from [CDC portal](https://www.atsdr.cdc.gov/place-health/php/svi/svi-data-documentation-download.html)

- Tract data for United States from 2022, 2020, 2010, 2000.  
- Data documentation

In [1]:
import ibis
from ibis import _
import streamlit as st
from utilities import generate_pmtiles

con = ibis.duckdb.connect("duck.db", extensions=['httpfs', 'spatial', 'h3'])


In [13]:
expr = con.read_geo("svi-data/2022/SVI2022_US_tract.gdb")
expr.to_parquet("svi-data/2022/SVI2022_US_tract.parquet")

# tippecanoe requires geojson input to create PMTiles.  Drop most additional variables in PMTiles creation.
query = ibis.to_sql(expr.select('STATE', 'COUNTY', 'LOCATION', 'FIPS', 'RPL_THEMES', 'Shape'))
con.raw_sql(f"COPY ({query}) TO '/tmp/svi.json' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');")

generate_pmtiles("/tmp/svi.json", "svi-data/2022/SVI2022_US_tract.pmtiles")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

For layer 0, using name "svi"
84120 features, 34922477 bytes of geometry, 5150225 bytes of string pool
tile 1/0/0 size is 673414 with detail 12, >500000    
Going to try keeping the sparsest 66.82% of the features to make it fit
tile 1/0/0 size is 654918 with detail 12, >500000    
Going to try keeping the sparsest 45.92% of the features to make it fit
tile 1/0/0 size is 627082 with detail 12, >500000    
Going to try keeping the sparsest 32.95% of the features to make it fit
tile 1/0/0 size is 571221 with detail 12, >500000    
Going to try keeping the sparsest 25.96% of the features to make it fit
tile 1/0/0 size is 515026 with detail 12, >500000    
Going to try keeping the sparsest 22.68% of the features to make it fit
tile 2/0/1 size is 556184 with detail 12, >500000    
Going to try keeping the sparsest 80.91% of the features to make it fit
tile 2/1/1 size is 680483 with detail 12, >500000    
Going to try keeping the sparsest 66.13% of the features to make it fit
tile 2/0/1 size

Successfully generated PMTiles file: svi-data/2022/SVI2022_US_tract.pmtiles


In [15]:
import minio
import re

minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
mc = minio.Minio("minio.carlboettiger.info", minio_key, minio_secret)

mc.fput_object("public-data", "social-vulnerability/2022/SVI2022_US_tract.pmtiles", "svi-data/2022/SVI2022_US_tract.pmtiles")
mc.fput_object("public-data", "social-vulnerability/2022/SVI2022_US_tract.parquet", "svi-data/2022/SVI2022_US_tract.parquet")


In [19]:



# Local cloud
minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
query1 =   f'''
CREATE OR REPLACE SECRET secret1 (
    TYPE S3,
    KEY_ID '{minio_key}',
    SECRET '{minio_secret}',
    ENDPOINT 'minio.carlboettiger.info',
    URL_STYLE 'path',
    SCOPE "s3://public-gbif"

);
'''
query2 =   f'''
CREATE OR REPLACE SECRET secret2 (
    TYPE S3,
    KEY_ID '{minio_key}',
    SECRET '{minio_secret}',
    ENDPOINT 'minio.carlboettiger.info',
    URL_STYLE 'path',
    SCOPE "s3://public-data"

);
'''
# don't scope to a single bucket
#     SCOPE 's3://public-gbif'

con.raw_sql(query1)
con.raw_sql(query2)
## Limits are sometimes good 
con.raw_sql("SET memory_limit = '20GB';")
con.raw_sql("set threads=40;")

# can/should we add explicit spatial index to gbif first?  using RTree takes too much memory

In [20]:
overture = (
    con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', 
                     filename=True, hive_partitioning=1))
usa = overture.filter(_.subtype=="country").filter(_.country == "US").select(_.geometry).execute()

In [21]:

gbif = con.read_parquet("s3://public-gbif/2024-10-01/**")
svi = con.read_parquet("s3://public-data/social-vulnerability/2022/SVI2022_US_tract.parquet").rename(geom = "Shape")


We iterate through the city list to do this efficiently.  (Should we filter gbif down to US boundary as a one-off first?  We will assume it is efficient to filter the full globe state by state)

In [23]:
all_states = svi.select(_.ST_ABBR).distinct().order_by(_.ST_ABBR).execute()["ST_ABBR"]
#all_states

In [26]:
## select from the list we haven't yet written (allows resume).
import minio
import re

minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
mc = minio.Minio("minio.carlboettiger.info", minio_key, minio_secret)
obj = mc.list_objects("public-gbif", "social-vulnerability", recursive=True)
pattern = r"social-vulnerability/|\.parquet$"
finished = [re.sub(pattern, "",  i.object_name) for i in obj if not i.is_dir]
remaining = set(all_states) - set(finished)

In [27]:
remaining

{'AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY'}

In [ ]:
## And here we go, long-running loop over each city
for i in remaining:
    counties = svi.filter(_.ST_ABBR == i).select(_.COUNTY).distinct().execute()["COUNTY"].to_numpy()
    for county in counties:
       gdf = (svi
              .filter(_.ST_ABBR == i, _.COUNTY== county)
              .mutate(area = _.geom.area())
       )

       print(i + "/" + county)
       
       bounds =  gdf.execute().total_bounds
       points = (gbif
              .filter(_.decimallongitude >= bounds[0], 
                     _.decimallongitude < bounds[2], 
                     _.decimallatitude >= bounds[1], 
                     _.decimallatitude < bounds[3])
              )
                     
       (gdf
       .join(points, gdf.geom.intersects(points.geom))
       .to_parquet(f"s3://public-gbif/social-vulnerability/state={i}/{county}.parquet")
       )


NV/Eureka County


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

NV/Lander County


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

NV/Clark County


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

NE/Kearney County


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

NE/Keith County


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

gbif_usa = con.read_parquet("s3://cboettig/gbif/svi/**")


In [43]:
gbif_usa

DatabaseTable: ibis_read_parquet_msislo4d7fcgdfh2pyoxvxjkdu
  OBJECTID                         int64
  ST                               string
  STATE                            string
  ST_ABBR                          string
  STCNTY                           string
  COUNTY                           string
  FIPS                             string
  LOCATION                         string
  AREA_SQMI                        float64
  E_TOTPOP                         int32
  M_TOTPOP                         int32
  E_HU                             int32
  M_HU                             int32
  E_HH                             int32
  M_HH                             int32
  E_POV150                         int32
  M_POV150                         int32
  E_UNEMP                          int32
  M_UNEMP                          int32
  E_HBURD                          int32
  M_HBURD                          int32
  E_NOHSDP                         int32
  M_NOHSDP                         int32
  E_UNINSUR                        int32
  M_UNINSUR                        int32
  E_AGE65                          int32
  M_AGE65                          int32
  E_AGE17                          int32
  M_AGE17                          int32
  E_DISABL                         int32
  M_DISABL                         int32
  E_SNGPNT                         int32
  M_SNGPNT                         int32
  E_LIMENG                         int32
  M_LIMENG                         int32
  E_MINRTY                         int32
  M_MINRTY                         int32
  E_MUNIT                          int32
  M_MUNIT                          int32
  E_MOBILE                         int32
  M_MOBILE                         int32
  E_CROWD                          int32
  M_CROWD                          int32
  E_NOVEH                          int32
  M_NOVEH                          int32
  E_GROUPQ                         int32
  M_GROUPQ                         int32
  EP_POV150                        float64
  MP_POV150                        float64
  EP_UNEMP                         float64
  MP_UNEMP                         float64
  EP_HBURD                         float64
  MP_HBURD                         float64
  EP_NOHSDP                        float64
  MP_NOHSDP                        float64
  EP_UNINSUR                       float64
  MP_UNINSUR                       float64
  EP_AGE65                         float64
  MP_AGE65                         float64
  EP_AGE17                         float64
  MP_AGE17                         float64
  EP_DISABL                        float64
  MP_DISABL                        float64
  EP_SNGPNT                        float64
  MP_SNGPNT                        float64
  EP_LIMENG                        float64
  MP_LIMENG                        float64
  EP_MINRTY                        float64
  MP_MINRTY                        float64
  EP_MUNIT                         float64
  MP_MUNIT                         float64
  EP_MOBILE                        float64
  MP_MOBILE                        float64
  EP_CROWD                         float64
  MP_CROWD                         float64
  EP_NOVEH                         float64
  MP_NOVEH                         float64
  EP_GROUPQ                        float64
  MP_GROUPQ                        float64
  EPL_POV150                       float64
  EPL_UNEMP                        float64
  EPL_HBURD                        float64
  EPL_NOHSDP                       float64
  EPL_UNINSUR                      float64
  SPL_THEME1                       float64
  RPL_THEME1                       float64
  EPL_AGE65                        float64
  EPL_AGE17                        float64
  EPL_DISABL                       float64
  EPL_SNGPNT                       float64
  EPL_LIMENG                       float64
  SPL_THEME2                       float64
  RPL_THEME2                       float64
  EPL_MINRTY              

In [ ]:
The four summary theme ranking variables, detailed in the Data Dictionary below, are:
• Socioeconomic Status - RPL_THEME1
• Household Characteristics - RPL_THEME2
• Racial & Ethnic Minority Status - RPL_THEME3
• Housing Type & Transportation - RPL_THEME4 

In [ ]:
import seaborn.objects as so

#df = gbif_usa.group_by(_.FIPS).agg(n = _.count().log(), svi = _.RPL_THEMES.mean()).execute()
df = gbif_usa.group_by(_.STATE, _.COUNTY).agg(n = _.count() / _.Shape_Area.sum(), svi1 = _.RPL_THEME1.mean(), svi3 = _.RPL_THEME3.mean()).execute()

so.Plot(df, x = "svi1", y="n", color = "svi3").add(so.Dots()).scale(y="log")

In [ ]:
import altair as alt

alt.Chart(df).mark_point().encode(
    x='svi1',
    y='n',
    color='svi3',
    tooltip = ['STATE', 'COUNTY']
)
